# Thermostat

In this example we have a look at a thermostat, which is a kind off discrete regulator. It switches a heating element on or of depending on defined upper and lower thresholds.

You can also find this example as a single file in the [GitHub repository](https://github.com/milanofthe/pathsim/blob/master/examples/examples_event/example_thermostat.py).

The continuous dynamics part of the system has the following two ODEs for the two heater states:

$$\begin{cases} 
\dot{T} = - a ( T - T_a ) + H & \text{heater on} \\
\dot{T} = - a ( T - T_a )     &  \text{heater off}
\end{cases}$$

With some algebraic manipulations we can translate the system equation into a block diagram that can be implemented in PathSim. Note the event manager, that watches the state of the integrator and controls the heater.

In [ ]:
import matplotlib.pyplot as plt

# Apply PathSim docs matplotlib style for consistent, theme-friendly figures
plt.style.use('../pathsim_docs.mplstyle')

from pathsim import Simulation, Connection
from pathsim.blocks import Integrator, Constant, Scope, Amplifier, Adder

# Event managers
from pathsim.events import ZeroCrossingUp, ZeroCrossingDown

Then lets define the system parameters.

In [ ]:
a = 0.3  # thermal capacity of room
Ta = 10  # ambient temperature
H = 5    # heater power
Kp = 25  # upper temperature threshold 
Km = 23  # lower temperature threshold

Now we can construct the continuous dynamic part of the system (its just a linear feedback system) by instantiating the blocks we need with their corresponding prameters and collect them together in a list:

In [ ]:
# Blocks that define the system
sco = Scope(labels=["temperature", "heater"])
integ = Integrator(Ta)
feedback = Amplifier(-a)
heater = Constant(H)
ambient = Constant(a*Ta)
add = Adder()

# Blocks of the main system
blocks = [sco, integ, feedback, heater, ambient, add]

In [ ]:
# The connections between the blocks
connections = [
    Connection(integ, feedback, sco),
    Connection(feedback, add),
    Connection(heater, add[1], sco[1]),
    Connection(ambient, add[2]),
    Connection(add, integ)
]

Next we need to implement the event managers for the threshold based switching between the two heater states.

In [ ]:
# Crossing upper threshold -> heater off

def func_evt_up(t):
    *_, x = integ()
    return x - Kp

def func_act_up(t):
    heater.off()

E1 = ZeroCrossingUp(
    func_evt=func_evt_up, 
    func_act=func_act_up
)


# Crossing lower threshold -> heater on

def func_act_down(t):
    heater.on()
 
def func_evt_down(t):
    *_, x = integ()
    return x - Km

E2 = ZeroCrossingDown(
    func_evt=func_evt_down, 
    func_act=func_act_down
)

events = [E1, E2]

In [ ]:
# Import the adaptive integrator to enable backtracking
from pathsim.solvers import RKBS32

# Initialize simulation 
Sim = Simulation(
    blocks,     
    connections, 
    events, 
    dt=0.1, 
    dt_max=0.05, 
    log=True, 
    Solver=RKBS32
)

Then we can run the simulation for some duration and see what happens.

In [ ]:
# Run simulation for some number of seconds
Sim.run(30)

In [ ]:
# Plot the results from the scope
fig, ax = sco.plot()
plt.show()

which looks like this:

There we can clearly see the switching of the heater and the room temperature oscillating between the upper and lower threshold. We can also add the events to the plot by just iterating the events to get the detected event times

In [ ]:
# Thermostat switching events - plot with event markers
fig, ax = sco.plot()

for e in E1: 
    ax.axvline(e, ls="--", c="#999999", alpha=0.7, label="Upper threshold" if e == list(E1)[0] else "")

for e in E2: 
    ax.axvline(e, ls="-.", c="#999999", alpha=0.7, label="Lower threshold" if e == list(E2)[0] else "")

ax.legend()
plt.show()

which looks like this: